In [ ]:
import pandas as pd
import requests
import random


from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.

  # Keys es útil para simular eventos de teclado en Selenium.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


In [ ]:
def from_csv_to_lists(csv):
  """
    Esta función toma una archivo csv y los pasa a lista de tuplas.
    
    Args:
    - csv: archivo csv.

    Returns:
    - list(df.itertuples(index=False, name=None)): lisat de tuplas
    """
  #lee el csv y lo pasa  a dataframe
  df = pd.read_csv(csv)
  try:
    df = df.drop('Unnamed: 0', axis=1)
  except:
    pass
  #devuelve lisat de tuplas
  return list(df.itertuples(index=False, name=None))

In [ ]:

    
#lista_imdb = [('tt0340084', 'The Drop', '2,7', 2006), ('tt5142104', 'Geocatching', 'NO DATA', 2015), ('tt1856101', 'Blade Runner 2049', '8,0', 2017), ('tt0106977', 'The Fugitive', '7,8', 1993)]

def puntuacion(lista_puntos_pelis):
    """
    Esta función toma una lista de tuplas de peliculas (id pelicula, titulo, puntuacion imdb, año) y crea una nueva lista de tuplas con las puntuaciones de imdb y rotten tomatoes.
    
    Args:
    - lista__puntos_pelis: lista de tuplas de tuplas de peliculas (id pelicula, titulo, puntuacion imdb, año).

    Returns:
    - lista_puntuacion: lista de tuplas de puntuaciones de peliculas (id pelicula, nombre pelicula, puntuaion imdb, puntuacion rotten tomatoes). 
    """
    driver = webdriver.Chrome()
    driver.maximize_window() 

    lista_puntuacion = []

    num = 1

    for peli in lista_puntos_pelis:

        driver.get("https://www.bing.com/")

        print(f"{num}. {peli[1]}")  
        
        sleep(1)
        
        #cookies
        try:
            driver.find_element("css selector", "#bnp_btn_reject > a").click()
        except:
            pass

        try:    
            driver.find_element("css selector", "#sb_form_q").send_keys(f'"{peli[1]}" "{peli[3]}" Rotten Tomatoes', Keys.ENTER)
        except:
            driver.get("https://www.bing.com/")
            sleep(0.5)
            driver.find_element("css selector", "#sb_form_q").send_keys(f'"{peli[1]}" "{peli[3]}" Rotten Tomatoes', Keys.ENTER)

        try:
            try:
                try:
                    valoracion = driver.find_element("css selector","#b_results > li:nth-child(1) > div.b_caption.hasdl.b_stsp2 > div > div > ul:nth-child(1) > li:nth-child(1) > div").text    
                except:
                    valoracion = driver.find_element("css selector",".b_sritem").text
            except:
                valoracion = driver.find_element("css selector","..cbl > div:nth-child(1) > div:nth-child(2)").text
                
            if '%' in valoracion:
                puntos_rotten = int(valoracion.split('%')[0])
                print(f"% - {puntos_rotten}")
            elif '/' in valoracion:    
                puntos_rotten = int(valoracion.split('/')[0])
                print(f"/ - {puntos_rotten}")
            else: puntos_rotten = 'NO DATA'
            
        except:
            puntos_rotten = 'NO DATA'        

        lista_puntuacion.append((peli[0], peli[1], peli[2], puntos_rotten))
        print(f"puntos rotten: {puntos_rotten}")
        num += 1
    driver.close()
    return lista_puntuacion  

In [ ]:
#lista de puntos imdb a rotten y devuelve una lisat final con las puntuaciones --> id, nombre peli, puntos imdb, puntos rotten --> a df y a csv
lista_puntos_imdb_2017_1000 = from_csv_to_lists('puntos_imdb_2017_1000.csv')
puntuaciones_2017_1000_100 = puntuacion(lista_puntos_imdb_2017_1000)
df_puntuaciones_2017_1000_100 = pd.DataFrame(puntuaciones_2017_1000_100)
df_puntuaciones_2017_1000_100.to_csv('puntuaciones_2017_1000_100.csv', index=False)